In [1]:
import datetime
import calendar
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sklearn.decomposition import FactorAnalysis
import statsmodels.api as sm

In [7]:
# factor analysis--replacing with regression
master_df = pd.read_excel('master_file.xlsx', 
                          sheet_name='Sheet1',
                          index_col='county_id'
                         )
master_na_df = master_df.drop(master_df.loc[master_df.isna().any(axis=1),:].index).copy()

# regression code


# factor analysis code

fa_col_list = ['distance_meat', 
               'distance_plant',
               'distance_foodhub', 
               'distance_hunting', 
               'food_banks',
               'food_insecurity_rate', 
               'pct_laccess_hhnv'
              ]

res = FactorAnalysis(n_components=1, 
                     rotation='varimax'
                    ).fit_transform(master_na_df[fa_col_list])

master_na_df['factor_score'] = res
master_na_df.to_excel('factor_scores.xlsx')
# negative factor scores are more food insecure?


In [6]:
master_na_df

,Unnamed: 0,distance_meat,distance_plant,distance_foodhub,distance_hunting,fips,food_banks,food_insecurity_rate,pct_laccess_hhnv,factor_score
county_id,,,,,,,,,,
1001,0,0.03,0.03,4.32,19.02,1001.0,0.0,0.157,0.033513,-0.050984
1003,1,0.23,0.02,0.13,0.38,1003.0,0.0,0.134,0.019051,-0.114796
1005,2,0.06,0.06,3.04,13.21,1005.0,0.0,0.207,0.043294,-0.070427
1007,3,0.67,0.67,13.54,12.29,1007.0,0.0,0.163,0.028214,0.037820
1009,4,0.69,0.69,3.53,17.98,1009.0,0.0,0.145,0.033364,-0.060318
...,...,...,...,...,...,...,...,...,...,...
56037,3138,13.87,13.87,38.12,3.07,56037.0,0.0,0.113,0.021828,0.284441
56039,3139,1.24,1.24,2.60,0.23,56039.0,0.0,0.090,0.005402,-0.089330
56041,3140,4.09,1.67,0.11,3.04,56041.0,0.0,0.126,0.027599,-0.111973


In [3]:
X = master_na_df[['distance_meat', 
                  'distance_plant', 
                  'distance_foodhub',
                  'distance_hunting', 
                  'food_banks',
                  'pct_laccess_hhnv']]
X = sm.add_constant(X)
y = master_na_df[['food_insecurity_rate']]
res = sm.OLS(y, X, missing='drop').fit()
display(res.summary())
master_na_df['food_insecurity_rate_pred'] = res.predict()
master_na_df['index'] = ((master_na_df['food_insecurity_rate_pred'] - 
                          master_na_df['food_insecurity_rate_pred'].mean()
                         )/master_na_df['food_insecurity_rate_pred'].std())
master_na_df.to_excel('index.xlsx')

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     food_insecurity_rate   R-squared:                       0.190
Model:                              OLS   Adj. R-squared:                  0.189
Method:                   Least Squares   F-statistic:                     122.7
Date:                  Sun, 09 Jul 2023   Prob (F-statistic):          1.19e-139
Time:                          17:13:05   Log-Likelihood:                 6225.1
No. Observations:                  3137   AIC:                        -1.244e+04
Df Residuals:                      3130   BIC:                        -1.239e+04
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.1133      0.001    116.720      0.000       0.111       0.115
distance_meat        0.0005      0.000      2.282      0.023    6.62e-05       0.001
distance_plant      -0.0010      0.000     -4.349      0.000      -0.002      -0.001
distance_foodhub     0.0002   4.85e-05      3.430      0.001    7.12e-05       0.000
distance_hunting    -0.0002   5.08e-05     -3.556      0.000      -0.000   -8.11e-05
food_banks          -0.0010      0.002     -0.473      0.636      -0.005       0.003
pct_laccess_hhnv     0.5485      0.021     26.737      0.000       0.508       0.589
==============================================================================
Omnibus:                       57.581   Durbin-Watson:                   1.057
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.775
Skew:                          -0.179   Prob(JB):                     3.20e-20
Kurtosis:                       3.747   Cond. No.                     4.17e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""